# Setup and Installations

In [17]:
import torch
import torchvision.models as models
from torch.profiler import profile, record_function, ProfilerActivity

# Model and Data Load

In [18]:
model = models.resnet18()
inputs = torch.randn(5, 3, 224, 224)

# CPU Profiling of the Model for Time

In [19]:
with profile(activities=[ProfilerActivity.CPU], record_shapes=True) as prof:
    with record_function("model_inference"):
        model(inputs)

In [20]:
print(prof.key_averages().table(sort_by="cpu_time_total", row_limit=10))


---------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  
                             Name    Self CPU %      Self CPU   CPU total %     CPU total  CPU time avg    # of Calls  
---------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  
                  model_inference         0.88%       3.134ms        99.99%     357.173ms     357.173ms             1  
                     aten::conv2d         0.02%      88.000us        74.64%     266.626ms      13.331ms            20  
                aten::convolution         0.09%     327.000us        74.62%     266.538ms      13.327ms            20  
               aten::_convolution         0.06%     219.000us        74.53%     266.211ms      13.311ms            20  
         aten::mkldnn_convolution        74.40%     265.761ms        74.47%     265.992ms      13.300ms            20  
                 aten::batch_norm       

In [21]:
print(prof.key_averages(group_by_input_shape=True).table(sort_by="cpu_time_total", row_limit=10))


---------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  --------------------------------------------------------------------------------  
                             Name    Self CPU %      Self CPU   CPU total %     CPU total  CPU time avg    # of Calls                                                                      Input Shapes  
---------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  --------------------------------------------------------------------------------  
                  model_inference         0.88%       3.134ms        99.99%     357.173ms     357.173ms             1                                                                                []  
                     aten::conv2d         0.01%      22.000us        17.96%      64.159ms      16.040ms             4                             [[5, 64, 56, 56], [64, 64, 3, 3], [], [], [], 

# CPU Profiling of the Model for Memory 

In [22]:
with profile(activities=[ProfilerActivity.CPU],
        profile_memory=True, record_shapes=True) as prof:
    model(inputs)

print(prof.key_averages().table(sort_by="self_cpu_memory_usage", row_limit=10))

---------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  
                             Name    Self CPU %      Self CPU   CPU total %     CPU total  CPU time avg       CPU Mem  Self CPU Mem    # of Calls  
---------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  
                      aten::empty         0.11%     411.000us         0.11%     411.000us       2.055us      94.36 Mb      94.36 Mb           200  
    aten::max_pool2d_with_indices         9.12%      33.561ms         9.12%      33.561ms      33.561ms      11.48 Mb      11.48 Mb             1  
                 aten::empty_like         0.03%     106.000us         0.06%     204.000us      10.200us      47.37 Mb     490.00 Kb            20  
                      aten::addmm         0.09%     339.000us         0.09%     349.000us     349.000us      19.

# GPU Profiling of the Model for Time

In [23]:
model = models.resnet18().cuda()
inputs = torch.randn(5, 3, 224, 224).cuda()

with profile(activities=[
        ProfilerActivity.CPU, ProfilerActivity.CUDA], record_shapes=True) as prof:
    with record_function("model_inference"):
        model(inputs)

print(prof.key_averages().table(sort_by="cuda_time_total", row_limit=10))

-------------------------------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  
                                                   Name    Self CPU %      Self CPU   CPU total %     CPU total  CPU time avg     Self CUDA   Self CUDA %    CUDA total  CUDA time avg    # of Calls  
-------------------------------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  
                                        model_inference        17.12%       2.109ms        80.03%       9.857ms       9.857ms       0.000us         0.00%      10.095ms      10.095ms             1  
                                           aten::conv2d         0.61%      75.000us        36.96%       4.552ms     227.600us       0.000us         0.00%       8.209ms     410.450us            20  
         

# GPU Profiling of the Model for Memory

In [24]:
with profile(activities=[ProfilerActivity.CPU, ProfilerActivity.CUDA],
        profile_memory=True, record_shapes=True) as prof:
    model(inputs)

print(prof.key_averages().table(sort_by="self_cpu_memory_usage", row_limit=10))

-------------------------------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  
                                                   Name    Self CPU %      Self CPU   CPU total %     CPU total  CPU time avg     Self CUDA   Self CUDA %    CUDA total  CUDA time avg       CPU Mem  Self CPU Mem      CUDA Mem  Self CUDA Mem    # of Calls  
-------------------------------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  
                                           aten::conv2d         0.97%     109.000us        46.29%       5.218ms     260.900us       0.000us         0.00%       8.227ms     411.350us           0 b           0 b      49.95 Mb           0 